In [1]:
import seaborn as sns
import pandas as pd
import requests
from fastai.vision.all import *
from ipywidgets import widgets

# Meet the world's first bear curator

The world is changing and museums are changing with it.

Challenging the status quo, our digital museum decided to hire a bear 
as Curator of Spanish Art.

Please join us to give a warm welcome **Oso Necesitis**

<img src="bear-curator.jpg" alt="drawing" width="200" align="left"/>

Leveraging the latest advances in artifical intelligence, Oso would be delighted to help you to identify Spanish paintings. 

However, you should know that Oso is still in training and there only knows the most famous Spanish painters (El Greco, Zurbarán, Ribera, Velázquez, Murillo, Goya, Sorolla, Picasso, Dalí, Miró)

## Try it yourself

Enter the URL (or upload a picture from your computer) and Oso will try to guess why major painter

In [2]:
FROM_URL = "From URL"
FROM_COMPUTER = "From Computer"

learn_inf = load_learner('classifier.pkl', cpu=True)


def choose_source(change):
    if w_source.value == FROM_URL:
        w_url_in.layout.visibility = "visible"
        w_btn_upload.layout.visibility = "hidden"
    elif w_source.value == FROM_COMPUTER:
        w_url_in.layout.visibility = "hidden"
        w_btn_upload.layout.visibility = "visible"

def get_picture(url:str):
    """ Grab URL picture and return results """
    r = requests.get(url)
    with open("to_predict.jpg", 'wb') as outfile:
        outfile.write(r.content)
    img = PILImage.create("to_predict.jpg")
    return img


def format_proba_table(df):
    """ Format with colors and sorted results"""
    cm = sns.light_palette('green', as_cmap=True)
    df = df.set_index("Artist")
    df = df.sort_values(by="Probability", ascending=False)
    df = df.style.background_gradient(cmap=cm)
    return df


def predict_on_click(change):
    """ Perform display picture + predict artist """
    # Download picture
    if w_source.value == FROM_COMPUTER:
        img = PILImage.create(w_btn_upload.data[-1])
    else:
        img = get_picture(w_url_in.value)
    # Display
    w_img.clear_output()
    with w_img: display(img.to_thumb(256,256))
    # Predict
    pred,pred_idx,probs = learn_inf.predict(img)
    w_result.value = f'The bear thinks this painting is from <b>{pred}</b>'
    # Widgets - prepare extra results
    w_btn_extra.layout.visibility = "visible"
    w_df.layout.visibility = "hidden"
    # Build Probability Table
    w_df.clear_output()
    df = pd.DataFrame({"Artist": learn_inf.dls.vocab, "Probability": probs})
    with w_df: display(format_proba_table(df))
    
def show_full_results(change):
    w_df.layout.visibility = "visible"

In [3]:
# Enter URL or upload file
w_source = widgets.ToggleButtons(options=[FROM_URL, FROM_COMPUTER])
w_url_in = widgets.Text(placeholder='Enter URL of picture to predict')
w_btn_upload = widgets.FileUpload()
w_btn_upload.layout.visibility = "hidden"
w_source.observe(choose_source)
# Display Image
w_img = widgets.Output()
# Predict Button
w_btn = widgets.Button(description="Ask the bear")
w_btn.on_click(predict_on_click)
# Result
w_result = widgets.HTML()
#widgets.HTML(value = f"<b><font color='red'>{text}</b>")
w_df = widgets.Output()
# Control Display full results
w_btn_extra = widgets.Button(description="See full results")
w_btn_extra.layout.visibility = "hidden"
w_btn_extra.on_click(show_full_results)

widgets.VBox([w_source, w_url_in, w_btn_upload, w_btn, w_result,  w_img, w_btn_extra, w_df])